# PM 2.5 Prediction

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from linear_regression import get_z_score_norm, LinearRegression, k_fold_cross_validation, get_lr_grid, z_score_norm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path, header=None)

## rename column names

In [3]:
train_df = train_df.rename(columns={"日期": "date", "測站": "station", "測項": "obs_item"})

In [4]:
train_df

,date,station,obs_item,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014-01-01,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014-01-01,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014-01-01,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014-01-01,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014-01-01,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014-12-20,豐原,THC,1.8,1.8,1.8,1.8,1.8,1.7,1.7,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,2014-12-20,豐原,WD_HR,46,13,61,44,55,68,66,...,59,308,327,21,100,109,108,114,108,109
4317,2014-12-20,豐原,WIND_DIREC,36,55,72,327,74,52,59,...,18,311,52,54,121,97,107,118,100,105
4318,2014-12-20,豐原,WIND_SPEED,1.9,2.4,1.9,2.8,2.3,1.9,2.1,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


In [5]:
test_df

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9
...,...,...,...,...,...,...,...,...,...,...,...
4315,id_239,THC,1.8,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.7
4316,id_239,WD_HR,80,92,95,95,96,97,96,96,84
4317,id_239,WIND_DIREC,76,99,93,97,93,94,98,97,65
4318,id_239,WIND_SPEED,2.2,3.2,2.5,3.6,5,4.2,5.7,4.9,3.6


# check missing value

In [6]:
train_df.isnull().values.any()

False

In [7]:
train_df[
    train_df.isna().any(axis=1)
]  # select all rows with NaN under an entire DataFrame:

,date,station,obs_item,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23


### Covert value columns to numeric

In [8]:
val_cols = train_df.columns[3:]
train_df[val_cols] = train_df[val_cols].apply(pd.to_numeric, errors="coerce")
train_df

,date,station,obs_item,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014-01-01,豐原,AMB_TEMP,14.00,14.00,14.00,13.00,12.00,12.00,12.00,...,22.00,22.00,21.00,19.00,17.00,16.00,15.00,15.00,15.00,15.00
1,2014-01-01,豐原,CH4,1.80,1.80,1.80,1.80,1.80,1.80,1.80,...,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80
2,2014-01-01,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.30,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014-01-01,豐原,NMHC,0.20,0.15,0.13,0.12,0.11,0.06,0.10,...,0.10,0.13,0.14,0.23,0.18,0.12,0.10,0.09,0.10,0.08
4,2014-01-01,豐原,NO,0.90,0.60,0.50,1.70,1.80,1.50,1.90,...,2.50,2.20,2.50,2.30,2.10,1.90,1.50,1.60,1.80,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014-12-20,豐原,THC,1.80,1.80,1.80,1.80,1.80,1.70,1.70,...,1.80,1.80,2.00,2.10,2.00,1.90,1.90,1.90,2.00,2.00
4316,2014-12-20,豐原,WD_HR,46.00,13.00,61.00,44.00,55.00,68.00,66.00,...,59.00,308.00,327.00,21.00,100.00,109.00,108.00,114.00,108.00,109.00
4317,2014-12-20,豐原,WIND_DIREC,36.00,55.00,72.00,327.00,74.00,52.00,59.00,...,18.00,311.00,52.00,54.00,121.00,97.00,107.00,118.00,100.00,105.00
4318,2014-12-20,豐原,WIND_SPEED,1.90,2.40,1.90,2.80,2.30,1.90,2.10,...,2.30,2.60,1.30,1.00,1.50,1.00,1.70,1.50,2.00,2.00


In [9]:
train_df.isnull().values.any()

True

In [10]:
train_df[train_df.isna().any(axis=1)]

,date,station,obs_item,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
10,2014-01-01,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2014-01-02,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,2014-01-03,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2014-01-04,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,2014-01-05,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240,2014-12-16,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4258,2014-12-17,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4276,2014-12-18,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4294,2014-12-19,豐原,RAINFALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,0.2,NaN,NaN,NaN,NaN,NaN,0.2,NaN,0.4


In [11]:
train_df[train_df.isna().any(axis=1)]["obs_item"].unique()

array(['RAINFALL'], dtype=object)

In [12]:
np.unique(train_df[train_df["obs_item"] == "RAINFALL"].iloc[:, 3:].values.flatten())

array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ,
        2.2,  2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,  3.8,  4. ,  4.2,
        4.6,  4.8,  5. ,  5.4,  6.4,  6.8,  7. ,  7.2,  7.4,  7.6,  7.8,
        8.2,  8.4,  8.6,  9.2,  9.8, 10. , 11. , 12. , 13. , 14. , 15. ,
       17. , 18. , 19. , 20. , 21. , 23. , 27. , 38. , 56. , 66. , 74. ,
        nan])

In [13]:
train_df.iloc[train_df[train_df["obs_item"] == "RAINFALL"].index] = train_df[train_df["obs_item"] == "RAINFALL"].fillna(0)

In [14]:
train_df.isnull().values.any()

False

In [15]:
train_df.iloc[:, 3:].values.dtype

dtype('float64')

# get rid of first 3 columns

In [16]:
train_data = train_df.iloc[:, 3:]

# make sure the observe item is the same order on every day

In [17]:
date_gb = train_df.groupby(["date"])
print(f"total day:\n{date_gb.size()} \n")

check_obs_item = []
for date, group in date_gb:
    check_obs_item.append(group["obs_item"].values)
np.unique(check_obs_item)

total day:
date
2014-01-01    18
2014-01-02    18
2014-01-03    18
2014-01-04    18
2014-01-05    18
              ..
2014-12-16    18
2014-12-17    18
2014-12-18    18
2014-12-19    18
2014-12-20    18
Length: 240, dtype: int64 



/tmp/ipykernel_115894/4004135284.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, group in date_gb:


array(['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
       'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC',
       'WIND_SPEED', 'WS_HR'], dtype=object)

### Uhmm, seems like the order is fine.

In [18]:
train_df[train_df["date"] == "2014-09-01"]["obs_item"]

2880      AMB_TEMP
2881           CH4
2882            CO
2883          NMHC
2884            NO
2885           NO2
2886           NOx
2887            O3
2888          PM10
2889         PM2.5
2890      RAINFALL
2891            RH
2892           SO2
2893           THC
2894         WD_HR
2895    WIND_DIREC
2896    WIND_SPEED
2897         WS_HR
Name: obs_item, dtype: object

# get year data

In [19]:
fc = 18  # feature count

year_data = list()
for month in range(12):  # 0 - 11
    total_hr = 24 * 20
    temp = np.zeros((fc, total_hr))

    day_per_month = 20
    for day in range(day_per_month):
        hr_idx = 24 * day
        row_idx = 18 * 20 * month + 18 * day
        temp[:, hr_idx : hr_idx + 24] = train_data.iloc[row_idx : row_idx + 18]

    year_data.append(temp)

year_data = np.array(year_data)
year_data.shape

(12, 18, 480)

# partition 9 hr interval as a x, and pm2.5 of 10th hr as a y

In [20]:
x_all, y_all = list(), list()

for month in range(12):
    month_data = year_data[month]
    for hr_itv_idx in range(24 * 20 - 9):
        x = month_data[:, hr_itv_idx : hr_itv_idx + 9]
        y = month_data[9, hr_itv_idx + 9]  # pm2.5 is at row-9

        x_all.append(x)
        y_all.append(y)

x_all = np.array(x_all)
y_all = np.array(y_all)

x_all.shape, y_all.shape

((5652, 18, 9), (5652,))

In [21]:
# 

In [22]:
norm_x, train_mean, train_std = get_z_score_norm(x_all)

### flatten x features

In [23]:
m, feat_n, hr_n, = norm_x.shape
norm_x = norm_x.reshape((m, -1))

norm_x.shape

(5652, 162)

### Train Valid Split

In [24]:
seed = 369
X_train, X_valid, y_train, y_valid = train_test_split(norm_x, y_all, test_size=0.2, random_state=seed)

# Learning Choose

In [25]:
# iteration = 10000
# lr_grid = get_lr_grid()
# lr_with_train_loss = []
# for lr in tqdm(lr_grid, total=len(lr_grid)):
#     model = LinearRegression(X_train, y_train, iteration, lr, validation=(X_valid, y_valid), verbose=False)
#     final_w, final_b, history = model.gradient_descent()
#     lr_with_train_loss.append((lr, history['train_loss']))

# for lr, hist in lr_with_train_loss:
#     plt.plot(np.arange(len(hist)), hist, label=f"{lr:.3e}")
#     plt.title('Training Loss with diff lr')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()

# Training

In [26]:
iteration = 20000
lr = 1.5

model = LinearRegression(X_train, y_train, iteration, lr, validation=(X_valid, y_valid))
# final_w, final_b, history = model.gradient_descent()
final_w, final_b, history = model.ada_grad()

Iteration    0: Cost 3514.733   
Iteration 2000: Cost   15.940   
Iteration 4000: Cost   15.715   
Iteration 6000: Cost   15.687   
Iteration 8000: Cost   15.680   
Iteration 10000: Cost   15.675   
Iteration 12000: Cost   15.671   
Iteration 14000: Cost   15.668   
Iteration 16000: Cost   15.665   
Iteration 18000: Cost   15.662   


# Predict Test Data

In [27]:
test_df

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9
...,...,...,...,...,...,...,...,...,...,...,...
4315,id_239,THC,1.8,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.7
4316,id_239,WD_HR,80,92,95,95,96,97,96,96,84
4317,id_239,WIND_DIREC,76,99,93,97,93,94,98,97,65
4318,id_239,WIND_SPEED,2.2,3.2,2.5,3.6,5,4.2,5.7,4.9,3.6


In [28]:
val_cols = test_df.columns[2:]
test_df[val_cols] = test_df[val_cols].apply(pd.to_numeric, errors='coerce')
test_df.iloc[test_df[test_df[1] == 'RAINFALL'].index] = test_df[test_df[1] == 'RAINFALL'].fillna(0)

In [29]:
def normalize(X: np.ndarray, mean, std):
    m, row, col = X.shape

    mean_2d = np.repeat(mean[:, np.newaxis], col, axis=1)  # (18, 9)
    mean_3d = np.repeat(mean_2d[np.newaxis, :, :], m, axis=0)  # (m, 18, 9)

    std_2d = np.repeat(std[:, np.newaxis], col, axis=1)  # (18, 9)
    std_3d = np.repeat(std_2d[np.newaxis, :, :], m, axis=0)  # (m, 18, 9)

    x_norm = (X - mean_3d) / std_3d

    return x_norm

In [30]:
temp = []
for g_name, g_df in test_df.groupby(0):
    x = g_df.iloc[:, 2:].values
    temp.append(x)
X_test = np.array(temp)
m, _, _, = X_test.shape

X_test_norm = normalize(X_test, train_mean, train_std).reshape((m, -1))
X_test_no_norm = X_test.reshape((m, -1))

In [31]:
pred = model.predict(X_test_norm)

In [32]:
test_ans_csv_path = 'data/ans.csv'
test_ans_df = pd.read_csv(test_ans_csv_path)

thtang_pred_csv_path = 'thtang_pred.csv'
thtang_pred_df = pd.read_csv(thtang_pred_csv_path)

In [33]:
score = np.sum(((test_ans_df['value'] - pred) ** 2) / len(pred))
print(f"score = {score}")

score = np.sum(((thtang_pred_df['value'] - pred) ** 2) / len(pred))
print(f"score = {score}")

score = np.sum(((test_ans_df['value'] - thtang_pred_df['value']) ** 2) / len(pred))
print(f"score = {score}")

score = 923.487840849066
score = 821.7644492509569
score = 837.6193240218593


In [34]:
close_form_pred = np.load('close_form_pred.npy')
score = np.sum(((close_form_pred - pred) ** 2) / len(pred))
print(f"score = {score}")

score = 771.269725107852


In [35]:
mean_squared_error(close_form_pred, pred)

771.269725107852